<a href="https://colab.research.google.com/github/jscienciadados/ciencia-dados/blob/main/avaliando-titulo-capitalizacao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt

# Conhecendo o Dataset

In [ ]:
df = pd.read_csv('train.csv', index_col='id')

In [ ]:
df.head(5)

In [ ]:
df.info()

In [ ]:
df.describe(include='object')

In [ ]:
plt.figure(figsize=(15,5))
sns.countplot(data=df, x='job')
plt.ylabel('Contagem')
plt.show()

In [ ]:
sns.barplot(data=df, x='marital', y='age', hue='y')
plt.show()

In [ ]:
sns.figsize=(15, 5) 
sns.catplot(data=df, x='y', y='campaign', aspect=8/6, height=8);

In [ ]:
df[df.y == 'yes'].describe()

In [ ]:
plt.figure(figsize=(15,5))
sns.scatterplot(x='age', y='duration', data=df, hue='y')
plt.show()

# Modelando

In [ ]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import CategoricalNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.externals import joblib
from sklearn import metrics

In [ ]:
# codificaçãão One-hot (mas não  vamos usar no modelo)
pd.get_dummies(df, columns=['job'])

In [ ]:
#Transforma as categorias em números
for c in df.columns[df.dtypes == object]: # df.dtypes == 'object'
    df[c] = df[c].astype('category')

In [ ]:
df.education.cat.reorder_categories(['illiterate', 'basic.4y', 'basic.6y', 'basic.9y', 'high.school', 
       'professional.course', 'university.degree'], ordered=True, inplace=True)

In [ ]:
#Transforma as categorias em números
for c in df.columns[df.dtypes == 'category']: # df.dtypes == 'object'
    df[c] = df[c].cat.codes

In [ ]:
df.head()

In [ ]:
y = df.y
X = df.drop('y', axis=1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42, shuffle=True, stratify=y)

In [ ]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape 

In [ ]:
# vericando a divisão
sum(y_train)/len(y_train), sum(y_test)/len(y_test)

In [ ]:
# Normalizar os dados
scaler = MinMaxScaler()
X_norm = scaler.fit_transform(X)

In [ ]:
df_norm = pd.DataFrame(X_norm, columns=df.columns.drop('y'))

In [ ]:
df_norm.head()

In [ ]:
knn_class = KNeighborsClassifier()
knn_class.fit(X_train, y_train)

In [ ]:
y_pred = knn_class.predict(X_test)

In [ ]:
print(metrics.classification_report(y_test, y_pred))

# Submetendo pro Kaggle

In [ ]:
aval = pd.read_csv('test.csv', index_col='id')
sub =  pd.read_csv('sample.csv', index_col='id')

In [ ]:
for c in aval.columns[aval.dtypes == object]: # df.dtypes == 'object'
    aval[c] = aval[c].astype('category')

In [ ]:
aval.education.cat.reorder_categories(['illiterate', 'basic.4y', 'basic.6y', 'basic.9y', 'high.school', 
       'professional.course', 'university.degree'], ordered=True, inplace=True)

In [ ]:
#Transforma as categorias em números
for c in aval.columns[aval.dtypes == 'category']: # df.dtypes == 'object'
    aval[c] = aval[c].cat.codes

In [ ]:
aval.head()

In [ ]:
X_aval = scaler.fit_transform(aval.values)

In [ ]:
aval_norm = pd.DataFrame(X_aval, columns=aval.columns)

In [ ]:
aval_norm.head()

In [ ]:
y_aval = knn_class.predict(X_aval)

In [ ]:
sub.y = pd.Series(y_aval)

In [ ]:
sub.head()

In [ ]:
sub.to_csv('submission.csv')

# Random Forest

In [ ]:
#Visualização do conjunto de treino
X.head()

In [ ]:
#pipeline para realizar os k-folds no conjunto de treino
pipeline = make_pipeline(StandardScaler(), 
                         RandomForestClassifier(n_estimators=200))
#grid de Hiperparâmetros para serem testados
hyperparameters = { 'randomforestclassifier__max_features' : ['auto', None, 'log2'],
                  'randomforestclassifier__max_depth': [None, 3, 1]}
# Grid para k-fold de 10 dobras
clf = GridSearchCV(pipeline, hyperparameters, cv=10)

In [ ]:
%%time
#Ajusta o modelo aos dados de treino
model=clf.fit(X,y)

In [ ]:
p = model.predict(test)

In [ ]:
#vizualização dos dados a serem submetidos
#sample.y= p
#sample.to_csv("~/submission.csv",index=False)
#sample.head()

# Métricas de avaliação

In [ ]:
solution = pd.read_csv('solution.csv').y
naive = pd.read_csv('allzeros.csv').y
rf = pd.read_csv('random_forest.csv').y

In [ ]:
# F1 de cada categoria
metrics.f1_score(solution, naive, pos_label=0), metrics.f1_score(solution, naive, pos_label=1)

In [ ]:
metrics.f1_score(solution, naive, average=None)

In [ ]:
metrics.f1_score(solution, naive, average='micro'), metrics.f1_score(solution, naive, average='macro')

In [ ]:
metrics.accuracy_score(solution,naive)

In [ ]:
metrics.precision_score(solution, naive, average=None), metrics.recall_score(solution, naive, average=None)

In [ ]:
metrics.precision_recall_fscore_support(solution, naive)

In [ ]:
print(metrics.classification_report(solution, rf))